In [1]:
from fabric import api as fapi

In [2]:
host = '129.114.108.248'

In [3]:
def assertRuns(*args, **kwargs):
    kwargs.setdefault('quiet', True)
    
    message = kwargs.pop('msg')
    with fapi.settings(warn_only=True):
        result = fapi.run(*args, **kwargs)
    if result.failed:
        raise AssertionError("{}\n---\noutput:\n{}".format(message, result))

In [4]:
with fapi.settings(user='cc', host_string=host):
    assertRuns('cc-checks --help', msg="cc-checks wasn't found/help not working")
    assertRuns('sudo cc-checks', msg="cc-checks doesn't run")
    assertRuns('cc-snapshot -h', msg="cc-snapshot wasn't found/help not working")
    assertRuns('etrace2 sleep 0.1', msg="etrace2 not working")

In [5]:
with fapi.settings(user='cc', host_string=host):
    assertRuns('nvidia-smi', msg="nvidia-smi not working")

In [13]:
import subprocess

cmd = ('ssh tacc "ssh -i .ssh/ccadmin '
                     '-o StrictHostKeyChecking=no '
                     '-o UserKnownHostsFile=/dev/null '
                     'ccadmin@{} whoami"').format(host)
proc = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, universal_newlines=True)
assert proc.returncode == 0
assert proc.stdout.strip() == 'ccadmin'

In [ ]:
with fapi.settings(user='root', host_string='tacc', use_ssh_config=True):
    fapi.run(('ssh -i .ssh/ccadmin '
              '-o StrictHostKeyChecking=no '
              '-o UserKnownHostsFile=/dev/null '
              'ccadmin@{} '
              'whoami').format(host))